In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
collections = [
    'cryptopunks',
    'boredapeyachtclub',
    'mutant-ape-yacht-club',
    'cryptoadz-by-gremplin',
    'neo-tokyo-identities',
    'doodles-official',
    'cool-cats-nft',
    'cyberkongz',
    'the-doge-pound',
    'meebits',
    # 'veefriends',
    'mekaverse',
    'creature-world-collection',
    'lazy-lions',
    'pudgypenguins',
    'deadfellaz',
    'sneaky-vampire-syndicate',
    'bored-ape-kennel-club',
    'world-of-women-nft',
    'supducks',
    # 'bears-deluxe',
    'the-sevens-official',
    'thehumanoids',
    'thecryptodads',
    'robotos-official'
    ]

collection_report = []

print(len(collections))

23


In [3]:
def getCollectionStats(collection):
    endpoint = 'https://api.opensea.io/api/v1/collection/'
    headers = {}

    response = requests.get(url=endpoint+collection)
    data = response.json()

    return data


def parseCollectionDetails(data):
    details = {
        'name': data['collection']['primary_asset_contracts'][0]['name'],
        'slug': data['collection']['slug'],
        'symbol': data['collection']['primary_asset_contracts'][0]['symbol'],
        'contract_address': data['collection']['primary_asset_contracts'][0]['address'],
        'payout_address': data['collection']['primary_asset_contracts'][0]['payout_address'],
        'total_supply': data['collection']['stats']['total_supply'],
        'contract_created_date': data['collection']['primary_asset_contracts'][0]['created_date']
        }

    return details


def parseTraits(data):
    attributes = list(data['collection']['traits'].keys())
    trait_counts = data['collection']['traits']
    
    return attributes, trait_counts


def buildShell(attributes):
    d = []

    for attribute in attributes:
        for trait, count in traits[attribute].items():
            results = {
                'Attribute': attribute,
                'Trait': trait,
                'Count': count,
            }
            d.append(results)

    return d

In [4]:
def calcFrequency(distribution):
    # Computes the Trait Frequency, Overall Frequency, and Attribute Frequency of the distribution
    for index, row in distribution.iterrows():
        distribution.loc[index, 'Trait_Frequency'] = row['Count'] / sum(distribution[distribution['Attribute']==row['Attribute']]['Count'])
        distribution.loc[index, 'Overall_Frequency'] = row['Count'] / parseCollectionDetails(data)['total_supply']
        distribution.loc[index, 'Attribute_Frequency'] = len(distribution[distribution['Attribute']==row['Attribute']]) / len(distribution['Attribute'])

    # distribution = distribution.assign(
    #     Trait_Frequency=lambda x: x['Count'] / sum(distribution[distribution['Attribute']==x['Attribute']]['Count']),
    #     Overall_Frequency=lambda x: x['Count'] / parseCollectionDetails(data)['total_supply'],
    #     Attribute_Frequency=lambda x: len(distribution[distribution['Attribute']==x['Attribute']]) / len(distribution['Attribute'])
    # )

In [5]:
def assignRanks(distribution, attributes):
    # Computes the Trait Rarity for each Trait within an Attribute in the distribution
    for attribute in attributes:
        trait_rank = {}
        subset = distribution[distribution['Attribute']==attribute]['Trait_Frequency'].drop_duplicates()
        rank = subset.sort_values().reset_index()
        for index, row in rank.iterrows():
            trait_rank[row['Trait_Frequency']] = index+1
        for index, row in distribution[distribution['Attribute']==attribute].iterrows():
            distribution.loc[index, 'Trait_Rank'] = trait_rank[row['Trait_Frequency']]

    # Computes the Trait Rarity across all Traits in the distribution
    distribution['Overall_Rank'] = list(map(lambda x: x+1, distribution['Overall_Frequency'].sort_values().reset_index().sort_values('index').index))

In [6]:
def calcStd(distribution):
    # Compute standard deviation of traits within an attribute
    for index, row in distribution.iterrows():
        distribution.loc[index, 'Attribute_std_dev'] = np.array(distribution[distribution['Attribute']==row['Attribute']]['Trait_Frequency']).std()

In [7]:
def calcRarityScore(distribution):
    # Computes the Rarity Score of each Trait in the distribution
    for index, row in distribution.iterrows():
        distribution.loc[index, 'Rarity_Score'] = row['Trait_Frequency'] * row['Overall_Frequency'] * row['Attribute_Frequency']

In [8]:
for collection in collections:
    print(f'Working on {collection}')
    data = getCollectionStats(collection)
    # print(data)
    collection_report.append(parseCollectionDetails(data))

    attributes, traits = parseTraits(data)
    d = buildShell(attributes)

    distribution = pd.DataFrame(d)

    calcFrequency(distribution)
    assignRanks(distribution, attributes)
    calcStd(distribution)
    calcRarityScore(distribution)
    distribution = distribution.assign(Symbol=parseCollectionDetails(data)['symbol'])
    distribution = distribution[['Symbol', 'Attribute', 'Trait', 'Count', 'Trait_Frequency', 'Trait_Rank', 'Overall_Frequency', 'Overall_Rank', 'Attribute_Frequency', 'Attribute_std_dev', 'Rarity_Score']]
    distribution.to_csv(f'distributions/{collection}_distribution.csv')

collections_out = pd.DataFrame(collection_report)
collections_out.to_csv('collections.csv')

Working on cryptopunks
Working on boredapeyachtclub
Working on mutant-ape-yacht-club
Working on cryptoadz-by-gremplin
Working on neo-tokyo-identities
Working on doodles-official
Working on cool-cats-nft
Working on cyberkongz
Working on the-doge-pound
Working on meebits
Working on mekaverse
Working on creature-world-collection
Working on lazy-lions
Working on pudgypenguins
Working on deadfellaz
Working on sneaky-vampire-syndicate
Working on bored-ape-kennel-club
Working on world-of-women-nft
Working on supducks
Working on the-sevens-official
Working on thehumanoids
Working on thecryptodads
Working on robotos-official
